# Install the snippet notebook

Store the snippet notebook on your Google Drive. Then copy the updated URL to 
Tools --> Settings --> Site --> Custom snippet notebook URL. Then when you open 
a new notebook, the snippets will be available.

1.   File --> Save Copy in Drive
2.   Tools --> Settings --> Site --> Custom snippet notebook URL


# Install cctbx (1/2)

This prodedure is adapted from Billy Poon's example: 
https://github.com/phenix-project/Colabs/blob/main/Start_cctbx.ipynb

The installation of cctbx involves a two step process:

1.   Install condacolab
2.   Install cctbx with mamba and move certain libraries to better locations.

The module condacolab install conda and mamba. Mamba is faster.
A progress bar will be updated as the snippet runs.

Completion of the first step will involve the generation of the error message:

**Your session crashed for an unknown reason.** 

Click on the X to close the error message 
before running **Install cctbx (2/2)**


In [ ]:
from IPython.utils import io
import tqdm.notebook

total = 50
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        !pip install -q condacolab
        pbar.update(10)

        import condacolab
        condacolab.install()
        pbar.update(40)   

# Install cctbx (2/2)

Use mamba to install cctbx-base.

In [ ]:

with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        !mamba install -q cctbx-base
        pbar.update(10)

        # conda installs ${CONDA_PREFIX}/share/cctbx into /usr/local instead of /usr
        !cp -af /usr/local/share/cctbx /usr/share/

        # Add libraries to the sys.path
        import sys
        for d in ['/usr/local/lib/python3.7/lib-dynload', '/usr/local/lib']:
          if d not in sys.path:
            sys.path.insert(0, d)
        pbar.update(10)

        # Check installation
        import os
        if os.path.isdir('/usr/local/share/cctbx') \
          and '/usr/local/lib/python3.7/lib-dynload' in sys.path \
          and '/usr/local/lib' in sys.path:
          print('Finished installing cctbx-base')
        else:
          raise RuntimeError('There was an error fixing up the installation of cctbx-base')
        pbar.update(30)

# Test cctbx

- The code prints three sets of parameter values below the code block.


In [ ]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=3.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

# Copy mtz files from Google Drive

Load mtz file(s) that you want to analyze onto Google Drive in a created-folder called `dataFiles` and then mount the Google Drive.



In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%shell cp ./drive/MyDrive/dataFiles/*.mtz .

# iotbx extractCrystalSymmetry

- Extract crystal symmetry from mtz file.
- To fetch the X-ray data in a cif file and convert to a mtz file, run the following command on your local computer after installing phenix.

```bash
phenix.fetch_pdb --mtz 3nd4
```

- The `$` below marks a site for editing.

```python
from __future__ import absolute_import, division, print_function
from iotbx import mtz
from cctbx import crystal

def extract_from(file_name):
  mtz_object = mtz.object(file_name=file_name)
  assert mtz_object.n_symmetry_matrices() > 0
  return mtz_object.crystals()[0].crystal_symmetry()
  
 extract_from(file_name="${1:3nd4}.mtz")
 ```



In [ ]:
from __future__ import absolute_import, division, print_function
from iotbx import mtz
from cctbx import crystal

def extract_from(file_name):
  mtz_object = mtz.object(file_name=file_name)
  assert mtz_object.n_symmetry_matrices() > 0
  return mtz_object.crystals()[0].crystal_symmetry()
  
extract_from(file_name="3nd4.mtz")


# iotbx fetchAtomcCif

- Fetch  atomic coordinates from RCSB in mmCIF format.
- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)


## iotbx fetchFASTA

- Fetch fasta file from RCSB.

- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)


# iotbx fetchPDB

- Fetch pdb file from RCSB in PDB format.

- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)


# iotbx fetchXrayCif

- Fetch X-ray data from RCSB in mmCIF format.
- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)


# iotbx symmetryFromPDB

- Print the symmetry from a PDB file.
- The `$` below marks a site for editing.

```python
import iotbx
iotbx.pdb.crystal_symmetry_from_pdb.extract_from("${1:3nd4}.pdb")
```


In [ ]:
import iotbx
iotbx.pdb.crystal_symmetry_from_pdb.extract_from("3nd4.pdb")

# millerArrays changeMtzColumns

- Read in mtz file from current directory and write out with fewer columns.
- The `$` below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("${1:/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz}")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="${2:FreeR_flag}")
mtz_dataset.mtz_object().write("${3:loxodata.mtz}")
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("ret_blu.mtz")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="FreeR_flag")
mtz_dataset.mtz_object().write("loxodata.mtz")

# millerArrays CNS2mtz

- Convert CNS reflection file into an mtz file.
- The `$` below marks a site for editing.

```python
from iotbx import reflection_file_reader
import os
reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars("${1:\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl}"))
from cctbx import crystal
crystal_symmetry = crystal.symmetry( unit_cell=(${2:97.37, 46.64, 65.47, 90, 115.4, 90}), space_group_symbol="${3:C2}")
miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)
mtz_dataset = None
for miller_array in miller_arrays:
    if (mtz_dataset is None):
        mtz_dataset = miller_array.as_mtz_dataset(
            column_root_label=miller_array.info().labels[0]) 
    else:
        mtz_dataset.add_miller_array(
            miller_array=miller_array, 
            column_root_label=miller_array.info().labels[0])
mtz_object = mtz_dataset.mtz_object() 
mtz_object.show_summary()
```

In [ ]:
from iotbx import reflection_file_reader
import os
reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars("\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl"))
from cctbx import crystal
crystal_symmetry = crystal.symmetry( unit_cell=(97.37, 46.64, 65.47, 90, 115.4, 90), space_group_symbol="C2")
miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)
mtz_dataset = None
for miller_array in miller_arrays:
    if (mtz_dataset is None):
        mtz_dataset = miller_array.as_mtz_dataset(
            column_root_label=miller_array.info().labels[0]) 
    else:
        mtz_dataset.add_miller_array(
            miller_array=miller_array, 
            column_root_label=miller_array.info().labels[0])
mtz_object = mtz_dataset.mtz_object() 
mtz_object.show_summary()

# millerArrays computeAllPossibleMillerIndices

- Compute all possible Miller indices.
- The `$` below marks a site for editing.

```python
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)
    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(${1:5.4307,5.4307,5.4307,90.00,90.0,90.00})
dmin=${2:1.0}
```

In [ ]:
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)

    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)
dmin=1.0


# millerArrays computeMillerIndicesInASU 

- Compute all possible Miller indices in the asymmetric (ASU).

- The `$`'s below marks a site for editing.

```bash
from cctbx import miller
import cctbx
from cctbx import crystal

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="${1:I432}",
        unit_cell=("2:{54.4,54.4,54.4,90.00,90.0,90.00}") ),
    anomalous_flag=${3:False},
    d_min=${4:0.4})

# map the reflections to the asu and print
  
msu = ms.map_to_asu()
[print(hkl2) for hkl2 in msu.indices()]
```

In [ ]:
from cctbx import miller
import cctbx
from cctbx import crystal

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="I432",
        unit_cell=("54.4,54.4,54.4,90.00,90.0,90.00") ),
    anomalous_flag=False,
    d_min=5.0)

for hkl in ms.indices():
    print(hkl)

# map the reflections to the asu and print
msu = ms.map_to_asu()
[print(hkl2) for hkl2 in msu.indices()]

# millerArrays IndicesInUnitCell

- Build miller indices given unit cell and resolution limit.

- The `$` below marks a site for editing.

In [6]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=10.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

(0, 0, 1)
(0, 0, 2)
(0, 0, 3)
(0, 0, 4)
(0, 0, 5)
(0, 0, 6)
(0, 0, 7)
(0, 0, 8)
(0, 0, 9)
(0, 0, 10)
(0, 0, 11)
(0, 0, 12)
(0, 0, 13)
(0, 0, 14)
(0, 0, 15)
(0, 0, 16)
(0, 0, 17)
(0, 0, 18)
(0, 0, 19)
(0, 0, 20)
(0, 0, 21)
(0, 0, 22)
(0, 0, 23)
(0, 0, 24)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 1, 3)
(0, 1, 4)
(0, 1, 5)
(0, 1, 6)
(0, 1, 7)
(0, 1, 8)
(0, 1, 9)
(0, 1, 10)
(0, 1, 11)
(0, 1, 12)
(0, 1, 13)
(0, 1, 14)
(0, 1, 15)
(0, 1, 16)
(0, 1, 17)
(0, 1, 18)
(0, 1, 19)
(0, 1, 20)
(0, 1, 21)
(0, 1, 22)
(0, 1, 23)
(0, 1, 24)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(0, 2, 3)
(0, 2, 4)
(0, 2, 5)
(0, 2, 6)
(0, 2, 7)
(0, 2, 8)
(0, 2, 9)
(0, 2, 10)
(0, 2, 11)
(0, 2, 12)
(0, 2, 13)
(0, 2, 14)
(0, 2, 15)
(0, 2, 16)
(0, 2, 17)
(0, 2, 18)
(0, 2, 19)
(0, 2, 20)
(0, 2, 21)
(0, 2, 22)
(0, 2, 23)
(0, 3, 0)
(0, 3, 1)
(0, 3, 2)
(0, 3, 3)
(0, 3, 4)
(0, 3, 5)
(0, 3, 6)
(0, 3, 7)
(0, 3, 8)
(0, 3, 9)
(0, 3, 10)
(0, 3, 11)
(0, 3, 12)
(0, 3, 13)
(0, 3, 14)
(0, 3, 15)
(0, 3, 16)
(0, 3, 17)
(0, 3, 18)
(0, 3, 19)
(0, 3, 20)
(0, 3

# millerArrays extractReflectionsInShell

- Extract the reflections in a shell.
- The `$` below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:3nd4}.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
for miller_array in miller_arrays:
    miller_array_truncated = miller_array.resolution_filter(d_min=${2:2}, d_max=${3:5})
print(miller_array_truncated)
 ```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="3nd4.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
for miller_array in miller_arrays:
    miller_array_truncated = miller_array.resolution_filter(d_min=2, d_max=5)
print(miller_array_truncated)


# millerArrays allIndicesInUnitCell
- Build miller indices given unit cell and resolution limit.
- The `$` below marks a site for editing.

```python
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="${1:P4}",
        unit_cell=(${2:150.8,150.8,250.4,90.0,90.0,90.0})),
    anomalous_flag=${3:False},
    d_min=${4:1.4})
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())
```

In [ ]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=1.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

# millerArrays printColumnLabelsMillerArray

```python
[print(f"Column label: '${1:key[2]}'")  for key in miller_arrays_dict.keys()]
```

In [1]:
[print(f"Column label: key[2]")  for key in miller_arrays_dict.keys()]

NameError: ignored

# millerArrays generateAllIndices

- Compute all possible Miller indices.
- The `$` below marks a site for editing.

```python
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)

    indices = []

    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(${1:5.4307,5.4307,5.4307,90.00,90.0,90.00})
dmin=${2:1.0}
```


In [ ]:
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)
    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)
dmin=1.0
generate_reflection_indices(uc=uc,dmin=dmin)


# millerArrays iotbx mtz2array

- Read in a mtz file into a Miller array with iotbx.file_reader.
- The `$` below marks a site for editing.

```python
from iotbx.file_reader import any_file
mtz_in = any_file("${1:data}.mtz", force_type="mtz")
miller_arrays = mtz_in.file_server.miller_arrays
```

In [ ]:
from iotbx.file_reader import any_file
mtz_in = any_file("3nd4.mtz", force_type="mtz")
miller_arrays = mtz_in.file_server.miller_arrays

# millerArrays normalizeStructureFactors

- Calculate quasi-normalized structure factor.
- The `$` below marks a site for editing.

In [ ]:
all_e_values = miller_array.quasi_normalize_structure_factors().sort(by_value="data")

# millerArrays truncate

- Read mtz file into a Miller array, truncate by resolution, and print summary.
- The `$` below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:3nd4}.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
miller_array_truncated = miller_arrays[0].resolution_filter(d_min=${2:2}, d_max=${3:5})
print(miller_array_truncated)
miller_array_truncated.show_summary()
```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="3nd4.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
miller_array_truncated = miller_arrays[0].resolution_filter(d_min=2, d_max=5)
print(miller_array_truncated)
miller_array_truncated.show_summary()

# millerArrays mtz2array

- Read a mtz file into a miller array.
- The `$` below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("${1:3hz7.mtz}")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("3hz7.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)

# millerArrays printColumnLabels

- Print column labels in a Miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().labels)) for i, miller_array in list(enumerate(miller_arrays))[:2]]

# millerArrays wavelength

- Print wavelengths of each miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().wavelength)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays source

- Print the source of each miller array.

In [ ]:
print("Miller Array %s: %s" % (i, miller_array.info().source)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays length

- Print length of miller arrays (i.e., the number of datasets in a mtz file).

In [ ]:
len(miller_arrays)

# millerArrays symmetry

- Print the crystal symmetry of each miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().crystal_symmetry_from_file)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays printHKLs

- Print all of the miller indices for a given Miller array.

In [ ]:
[print(hkl) for hkl in miller_arrays[0].indices()]

# millerArrays CCone-half

- Return CC one-half for a given Miller array. 
- The `$` below marks a site for editing.

```python
miller_arrays[${1:0}].cc_one_half()
```


In [ ]:
miller_arrays[0].cc_one_half()

# millerArrays dstar

- Return the resolution range in d* in a specified Miller array.
- The `$` below marks a site for editing.

```python
miller_arrays[${1:0}].min_max_d_star_sq()
```

In [ ]:
miller_arrays[0].min_max_d_star_sq()

# millerArrays DminDmax

- Return the resolution range in Angstroms for a Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].d_max_min()
```

In [ ]:
miller_arrays[0].d_max_min()

# millerArrays IoverSig

- Return the I/sig overall for a given Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].i_over_sig_i()
```

In [ ]:
miller_arrays[0].i_over_sig_i()

# millerArrays CConeHalfSigmaTau

- Return CC one-half sigma tau for a given Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].cc_one_half_sigma_tau()
```

In [ ]:
miller_arrays[0].cc_one_half_sigma_tau()

# millerArrays CConeHalf

- Return CC one-half for a given Miller array.
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].cc_one_half()
```

In [3]:
miller_arrays[0].cc_one_half()

NameError: ignored

# millerArrays bijvoetRatios

- Print the Bijvoet ratios in a specified Miller array.  May have to average by bin first.
- The $ below marks a site for editing.

```python
[print(i) for i in miller_arrays[${1:0}].bijvoet_ratios()]
```

In [ ]:
[print(i) for i in miller_arrays[0].bijvoet_ratios()]

# millerArrays measurability

- Return the `measurability` of the anomalous signal in a specified Miller array.
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].measurability()
```

In [ ]:
miller_arrays[0].measurability()

# millerArrays anomalousSignal

- Return the anomalous signal in a specified Miller array.
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].anomalous_signal()
```

In [ ]:
miller_arrays[0].anomalous_signal()

# millerArrays comprehensiveSummary

- Show comprehensive summary for a specified Miller array.
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].show_comprehensive_summary()
```

In [ ]:
miller_arrays[0].show_comprehensive_summary()

# millerArrays numberBijvoetArrays

- Show number of bijvoet pairs for a specified Miller array.
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].n_bijvoet_pairs()
```

In [ ]:
miller_arrays[0].n_bijvoet_pairs()

# millerArrays wilsonRatio

- Show wilson ratio of miller array for a specified Miller array. 
- The $ below marks a site for editing.

```python
miller_arrays[${1:0}].wilson_ratio()
```

In [ ]:
miller_arrays[0].wilson_ratio()

# millerArrays IpIn

- Unpack into I(+) and I(-) for a specified Miller array. 
- The $ below marks a site for editing.

```python
Iobs = miller_arrays[${1:0}]
i_plus, i_minus = Iobs.hemispheres_acentrics()
ipd = i_plus.data()
ip=list(ipd)
imd = i_minus.data()
im = list(imd)
len(im)
Iobs.show_summary()
print(Iobs.info())
print(Iobs.observation_type())
```

In [ ]:
Iobs = miller_arrays[0]
i_plus, i_minus = Iobs.hemispheres_acentrics()
ipd = i_plus.data()
ip=list(ipd)
imd = i_minus.data()
im = list(imd)
len(im)
Iobs.show_summary()
print(Iobs.info())
print(Iobs.observation_type())

# millerArrays printSelectRows

- Print five rows of the Iobs for a specified Miller array. 
- The $ below marks a site for editing.

```python
list(Iobs[${1:100:105}])
```

In [ ]:
list(Iobs[100:105])

# millerArrays extractIntensities

- Extract just the intensities for a give Miller array and print ten rows of them.
- The $ below marks a site for editing.

```python
Iobs = miller_arrays[${1:0}]
iobsdata = Iobs.data()
list(iobsdata[${1:100:110}])
```

In [ ]:
Iobs = miller_arrays[0]
iobsdata = Iobs.data()
list(iobsdata[100:110])

# millerArrays convert2MTZ

- Convert a miller array into a mtz_dataset and write out as a mtz file.
- The $ below marks a site for editing.

```python
mtz_dataset = Iobs.as_mtz_dataset(column_root_label="${1:I}")
mtz_dataset.mtz_object().write("${2:3hz7intensities}.mtz")
```

In [ ]:
mtz_dataset = Iobs.as_mtz_dataset(column_root_label="I")
mtz_dataset.mtz_object().write("3hz7intensities.mtz")

# millerArrays printIntensities

- Print all of the intensities for a given Miller array.


In [ ]:
[print(hkl) for hkl in miller_arrays[1].data()]

# millerArrays 

- Read in the mtz file and print its column labels as a sanity check.
- The $ below marks a site for editing.

```python
mtz_filename2 = "${1:3hz7intensities}.mtz"
mtz_file2 = mtz.object(mtz_filename2)
mtz_file2.column_labels()
```

In [ ]:
mtz_filename2 = "3hz7intensities.mtz"
mtz_file2 = mtz.object(mtz_filename2)
mtz_file2.column_labels()


# millerArrays array2Dictionary

- Set up the arrays as dictionaries.
- The $ below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:3nd4}.mtz")
# Only works with mtz.object. 
# Does not work if mtz is read in with iotbx.file_reader.
miller_arrays_dict = mtz_obj.as_miller_arrays_dict()
```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="3nd4.mtz")
# Only works with mtz.object. 
# Does not work if mtz is read in with iotbx.file_reader.
miller_arrays_dict = mtz_obj.as_miller_arrays_dict()

# millerArrays dictKeys

- Print the miller keys() of a miller dictionary.


In [ ]:
miller_arrays_dict.keys()

# millerArrays printColumnsMillerDict

- Print the column labels of Miller dictionary.
- The $ below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:/Users/blaine/3nd4.mtz}")
# Only works with mtz.object. Does not work if mtz is read in with iotbx.file_reader.
miller_arrays_dict = mtz_obj.as_miller_arrays_dict()
[print(f"Column label: {key[2]}")  for key in miller_arrays_dict.keys()]
```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="/Users/blaine/3nd4.mtz")
# Only works with mtz.object. Does not work if mtz is read in with iotbx.file_reader.
miller_arrays_dict = mtz_obj.as_miller_arrays_dict()
[print(f"Column label: {key[2]")  for key in miller_arrays_dict.keys()]

# millerArrays changeMTZcolumns

- Read in mtz file and write out with fewer columns.
- The $ below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("${1:/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz}")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="${2:FreeR_flag}")
mtz_dataset.mtz_object().write("${3:loxodata.mtz}")
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="FreeR_flag")
mtz_dataset.mtz_object().write("loxodata.mtz")

# millerArrays generateLplot

- The $ below marks a site for editing.
- Use pandas to read in a hkl file with whitespace separators into a dataframe.
- Append to the dataframe a column with F/sigmaF values.
- Append to the dataframe a column with the absolute value of the L indices.
- Average F/sigmaF by absL index.
- Write to absL and F/sigmaF to csv file.

```python
mtzdata = pd.read_csv("${1:1524start}.hkl", names=["H","K","L",${2:"F","SIGF"}], sep="\s+")
mtzdata["FovSigF"] = mtzdata.apply(lambda row: row["F"] / row["SIGF"], axis=1)
mtzdata["absL"] = mtzdata.apply(lambda row: abs(row["L"]), axis=1)
FovSigFabsL = mtzdata.groupby([mtzdata.absL]).FovSigF.mean()
FovSigFabsL.to_csv("${3:test2}.csv")
```

In [ ]:
mtzdata = pd.read_csv("1524start.hkl", names=["H","K","L","F","SIGF"], sep="\s+")
mtzdata["FovSigF"] = mtzdata.apply(lambda row: row["F"] / row["SIGF"], axis=1)
mtzdata["absL"] = mtzdata.apply(lambda row: abs(row["L"]), axis=1)
FovSigFabsL = mtzdata.groupby([mtzdata.absL]).FovSigF.mean()
FovSigFabsL.to_csv("test2.csv")

# xrayDataPlots IpIn_scatterplot

- Scatter plot of I(+) and (I(-). The presence of an anomalous signal is indicated by deviations from x=y.
- The $ below marks a site for editing.

```python
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator #, FormatStrFormatter
from matplotlib.ticker import FuncFormatter
from iotbx.reflection_file_reader import any_reflection_file

# >>> change the mtz file name
hkl_file = any_reflection_file("${1:3hz7}.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)
Iobs = miller_arrays[1]
i_plus, i_minus = Iobs.hemispheres_acentrics()
ipd = i_plus.data()
ip=list(ipd)
imd = i_minus.data()
im = list(imd)
len(im)

comma_fmt = FuncFormatter(lambda x, p: format(int(x), ","))

mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["figure.dpi"] = 600

# Set to width of a one column on a two-column page.
# May want to adjust settings for a slide.
fig, ax = plt.subplots(figsize=[3.25, 3.25])
ax.scatter(ip,im,c="k",alpha=0.3,s=5.5)
ax.set_xlabel(r"I(+)",fontsize=12)
ax.set_ylabel(r"I(-)",fontsize=12)
ax.xaxis.set_major_locator(MultipleLocator(50000.))
ax.yaxis.set_major_locator(MultipleLocator(50000.))
ax.get_xaxis().set_major_formatter(comma_fmt)
ax.get_yaxis().set_major_formatter(comma_fmt)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(False)

# >>> change name of the figure file
plt.savefig("${1:3hz7}IpIm.pdf",bbox_inches="tight")
```

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator #, FormatStrFormatter
from matplotlib.ticker import FuncFormatter
from iotbx.reflection_file_reader import any_reflection_file

# >>> change the mtz file name
hkl_file = any_reflection_file("3hz7.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)
Iobs = miller_arrays[1]
i_plus, i_minus = Iobs.hemispheres_acentrics()
ipd = i_plus.data()
ip=list(ipd)
imd = i_minus.data()
im = list(imd)
len(im)

comma_fmt = FuncFormatter(lambda x, p: format(int(x), ","))

mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["figure.dpi"] = 600

# Set to width of a one column on a two-column page.
# May want to adjust settings for a slide.
fig, ax = plt.subplots(figsize=[3.25, 3.25])
ax.scatter(ip,im,c="k",alpha=0.3,s=5.5)
ax.set_xlabel(r"I(+)",fontsize=12)
ax.set_ylabel(r"I(-)",fontsize=12)
ax.xaxis.set_major_locator(MultipleLocator(50000.))
ax.yaxis.set_major_locator(MultipleLocator(50000.))
ax.get_xaxis().set_major_formatter(comma_fmt)
ax.get_yaxis().set_major_formatter(comma_fmt)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(False)

# >>> change name of the figure file
plt.savefig("3hz7IpIm.pdf",bbox_inches="tight")

# xrayDataPlot meanIvsDstar

- Miller arrays to plot of bin mean intensity over dstar.
- The $ below marks a site for editing.

```python
# Description:  Miller arrays to plot of bin mean intensity over dstar
# Source:  NA

"""
from iotbx.file_reader import any_file
import matplotlib.pyplot as plt

f = any_file("${1:/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz}")

print(f.file_type)
f.show_summary()
miller_arrays = f.file_server.miller_arrays
iobs =  miller_arrays[3]
flags = miller_arrays[0]
iobs, flags = iobs.common_sets(other=flags)
iobsData = iobs.data()
list(iobsData[100:110])
iobs.show_comprehensive_summary()
# iobs.binner()
n_bins = ${2:20}
binner = iobs.setup_binner(n_bins=n_bins)
binner.show_summary()

used = list(binner.range_used())
selections = [binner.selection(i) for i in used]
means = [iobs.select(sel).mean() for sel in selections]

from math import log
lnmeans = [log(y) for y in means]

d_star_power = 1.618034
centers = binner.bin_centers(d_star_power)
d_centers = list(centers**(-1 / d_star_power))
d_centers

# plt.ylabel("Natural log of the amplitudes squared")
# plt.xlabel(r"$\textrm{d^*}$ in $\textrm{\AA}$")
# ax.set_xlim(35, 1.5)
# plt.scatter(d_centers,lnmeanss)

fig, ax = plt.subplots()
ax.scatter(d_centers,lnmeans)
ax.set_xlim(${3:8}, ${4:1.5})  # decreasing
ax.set_xlabel(r"$d^*$ in $\AA$")
ax.set_ylabel("Natural log of the intensities")
ax.grid(False)
plt.savefig("${5:iobsvsdstar}.pdf")
```

In [ ]:
from iotbx.file_reader import any_file
import matplotlib.pyplot as plt

f = any_file("/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz")

print(f.file_type)
f.show_summary()
miller_arrays = f.file_server.miller_arrays
iobs =  miller_arrays[3]
flags = miller_arrays[0]
iobs, flags = iobs.common_sets(other=flags)
iobsData = iobs.data()
list(iobsData[100:110])
iobs.show_comprehensive_summary()
# iobs.binner()
n_bins = 20
binner = iobs.setup_binner(n_bins=n_bins)
binner.show_summary()

used = list(binner.range_used())
selections = [binner.selection(i) for i in used]
means = [iobs.select(sel).mean() for sel in selections]

from math import log
lnmeans = [log(y) for y in means]

d_star_power = 1.618034
centers = binner.bin_centers(d_star_power)
d_centers = list(centers**(-1 / d_star_power))
d_centers

# plt.ylabel("Natural log of the amplitudes squared")
# plt.xlabel(r"$\textrm{d^*$ in $\textrm{\AA$")
# ax.set_xlim(35, 1.5)
# plt.scatter(d_centers,lnmeanss)

fig, ax = plt.subplots()
ax.scatter(d_centers,lnmeans)
ax.set_xlim(8, 1.5)  # decreasing
ax.set_xlabel(r"$d^*$ in $\AA$")
ax.set_ylabel("Natural log of the intensities")
ax.grid(False)

# xrayDataPlot FcalcsVsResolutionBin

- Example of computing Fcalcs and then plotting them by resolution bin. This script uses miller arrays and binner.
- The $ below marks a site for editing.
- This script reads in a phenix.refine mtz file.
- It plots the R-factor by resolution bin.
- The plots are made with matplotlib using miller arrays.
- It also plots the correlation coefficients.
- The plots were made with matplotlib.
- This script was adapted from an example script in iotbx:  

Source:  https://github.com/cctbx/cctbx_project/blob/master/
iotbx/examples/recalculate_phenix_refine_r_factors.py


```python
# get_ipython().run_line_magic("matplotlib", "inline")

from __future__ import absolute_import, division, print_function
from iotbx.reflection_file_utils import get_r_free_flags_scores
from iotbx.file_reader import any_file
import matplotlib
import matplotlib.pyplot as plt



def compute_r_factors(fobs, fmodel, flags):
  fmodel, fobs = fmodel.common_sets(other=fobs)
  fmodel, flags = fmodel.common_sets(other=flags)
  fc_work = fmodel.select(~(flags.data()))
  fo_work = fobs.select(~(flags.data()))
  fc_test = fmodel.select(flags.data())
  fo_test = fobs.select(flags.data())
  r_work = fo_work.r1_factor(fc_work)
  r_free = fo_test.r1_factor(fc_test)
  
  print("r_work = %.4f" % r_work)
  print("r_free = %.4f" % r_free)
  print("")

  binner = flags.setup_binner(n_bins=20)
  d_star_power = 1.618034
  centers = binner.bin_centers(d_star_power)
  d_centers = list(centers**(-1 / d_star_power))
#   for i in d_centers:
#     print(i)
    
  fo_work.use_binning_of(flags)
  fc_work.use_binner_of(fo_work)
  fo_test.use_binning_of(fo_work)
  fc_test.use_binning_of(fo_work)

  r_work_list = []
  r_free_list = []
  cc_work_list = []
  cc_free_list = []
  for i_bin in fo_work.binner().range_all():
    sel_work = fo_work.binner().selection(i_bin)
    sel_test = fo_test.binner().selection(i_bin)
    fo_work_bin = fo_work.select(sel_work)
    fc_work_bin = fc_work.select(sel_work)
    fo_test_bin = fo_test.select(sel_test)
    fc_test_bin = fc_test.select(sel_test)
    if fc_test_bin.size() == 0 : continue
        
    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,
      assume_index_matching=True)
    r_work_list.append(r_work_bin)
    
    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,
      assume_index_matching=True)
    r_free_list.append(r_free_bin)
    
    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()
    cc_work_list.append(cc_work_bin)
    
    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()
    cc_free_list.append(cc_free_bin)
    
    legend = flags.binner().bin_legend(i_bin, show_counts=False)
    print("%s  %8d %8d  %.4f %.4f  %.3f %.3f" % (legend, fo_work_bin.size(),
      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))
    
  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list


def plot_r_factors(d_centers, r_work_list, r_free_list):
  plt.scatter(d_centers, r_work_list, label=r"$\mathit{R_{work}}$")
  plt.scatter(d_centers, r_free_list, label=r"$\mathit{R_{free}}$")
  plt.xlabel(r"Resolution ($\mathrm{\AA}$)")
  plt.ylabel(r"R-factor (%)")
  plt.legend(loc="upper right")
  plt.savefig("Rs.pdf")
  plt.close()


def plot_cc(d_centers, cc_work_list, cc_free_list):
  plt.scatter(d_centers, cc_work_list, label=r"$\mathit{CC_{work}}$")
  plt.scatter(d_centers, cc_free_list, label=r"$\mathit{CC_{free}}$")
  plt.xlabel(r"Resolution ($\mathrm{\AA}$)")
  plt.ylabel(r"Correlation Coefficeint Fo vs Fc (%)")
  plt.legend(loc="lower right")
  plt.savefig("CCs.pdf")


def run(input_mtz):
  mtz_in = any_file(input_mtz)
  ma = mtz_in.file_server.miller_arrays
  flags = fmodel = fobs = None
  # select the output arrays from phenix.refine.  This could easily be modified
  # to handle MTZ files from other programs.
  for array in ma :
    labels = array.info().label_string()
    if labels.startswith("R-free-flags"):
      flags = array
    elif labels.startswith("F-model"):
      fmodel = abs(array)
    elif labels.startswith("F-obs-filtered"):
      fobs = array
  if (None in [flags, fobs, fmodel]):
    raise RuntimeError("Not a valid phenix.refine output file")
  scores = get_r_free_flags_scores([flags], None)
  test_flag_value = scores.test_flag_values[0]
  flags = flags.customized_copy(data=flags.data()==test_flag_value)

  (d_centers, 
   r_work_list, 
   r_free_list, 
   cc_work_list, 
   cc_free_list) = compute_r_factors(fobs, fmodel, flags)
  plot_r_factors(d_centers, r_work_list, r_free_list)
  plot_cc(d_centers, cc_work_list, cc_free_list)


if (__name__ == "__main__"):
  run(input_mtz="${1:28molrepEdited_5_refine_001}.mtz")
```

In [ ]:
# get_ipython().run_line_magic("matplotlib", "inline")

from __future__ import absolute_import, division, print_function
from iotbx.reflection_file_utils import get_r_free_flags_scores
from iotbx.file_reader import any_file
import matplotlib
import matplotlib.pyplot as plt



def compute_r_factors(fobs, fmodel, flags):
  fmodel, fobs = fmodel.common_sets(other=fobs)
  fmodel, flags = fmodel.common_sets(other=flags)
  fc_work = fmodel.select(~(flags.data()))
  fo_work = fobs.select(~(flags.data()))
  fc_test = fmodel.select(flags.data())
  fo_test = fobs.select(flags.data())
  r_work = fo_work.r1_factor(fc_work)
  r_free = fo_test.r1_factor(fc_test)
  
  print("r_work = %.4f" % r_work)
  print("r_free = %.4f" % r_free)
  print("")

  binner = flags.setup_binner(n_bins=20)
  d_star_power = 1.618034
  centers = binner.bin_centers(d_star_power)
  d_centers = list(centers**(-1 / d_star_power))
#   for i in d_centers:
#     print(i)
    
  fo_work.use_binning_of(flags)
  fc_work.use_binner_of(fo_work)
  fo_test.use_binning_of(fo_work)
  fc_test.use_binning_of(fo_work)

  r_work_list = []
  r_free_list = []
  cc_work_list = []
  cc_free_list = []
  for i_bin in fo_work.binner().range_all():
    sel_work = fo_work.binner().selection(i_bin)
    sel_test = fo_test.binner().selection(i_bin)
    fo_work_bin = fo_work.select(sel_work)
    fc_work_bin = fc_work.select(sel_work)
    fo_test_bin = fo_test.select(sel_test)
    fc_test_bin = fc_test.select(sel_test)
    if fc_test_bin.size() == 0 : continue
        
    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,
      assume_index_matching=True)
    r_work_list.append(r_work_bin)
    
    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,
      assume_index_matching=True)
    r_free_list.append(r_free_bin)
    
    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()
    cc_work_list.append(cc_work_bin)
    
    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()
    cc_free_list.append(cc_free_bin)
    
    legend = flags.binner().bin_legend(i_bin, show_counts=False)
    print("%s  %8d %8d  %.4f %.4f  %.3f %.3f" % (legend, fo_work_bin.size(),
      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))
    
  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list


def plot_r_factors(d_centers, r_work_list, r_free_list):
  plt.scatter(d_centers, r_work_list, label=r"$\mathit{R_{work$")
  plt.scatter(d_centers, r_free_list, label=r"$\mathit{R_{free$")
  plt.xlabel(r"Resolution ($\mathrm{\AA$)")
  plt.ylabel(r"R-factor (%)")
  plt.legend(loc="upper right")
  plt.savefig("Rs.pdf")
  plt.close()


def plot_cc(d_centers, cc_work_list, cc_free_list):
  plt.scatter(d_centers, cc_work_list, label=r"$\mathit{CC_{work$")
  plt.scatter(d_centers, cc_free_list, label=r"$\mathit{CC_{free$")
  plt.xlabel(r"Resolution ($\mathrm{\AA$)")
  plt.ylabel(r"Correlation Coefficeint Fo vs Fc (%)")
  plt.legend(loc="lower right")
  plt.savefig("CCs.pdf")


def run(input_mtz):
  mtz_in = any_file(input_mtz)
  ma = mtz_in.file_server.miller_arrays
  flags = fmodel = fobs = None
  # select the output arrays from phenix.refine.  This could easily be modified
  # to handle MTZ files from other programs.
  for array in ma :
    labels = array.info().label_string()
    if labels.startswith("R-free-flags"):
      flags = array
    elif labels.startswith("F-model"):
      fmodel = abs(array)
    elif labels.startswith("F-obs-filtered"):
      fobs = array
  if (None in [flags, fobs, fmodel]):
    raise RuntimeError("Not a valid phenix.refine output file")
  scores = get_r_free_flags_scores([flags], None)
  test_flag_value = scores.test_flag_values[0]
  flags = flags.customized_copy(data=flags.data()==test_flag_value)

  (d_centers, 
   r_work_list, 
   r_free_list, 
   cc_work_list, 
   cc_free_list) = compute_r_factors(fobs, fmodel, flags)
  plot_r_factors(d_centers, r_work_list, r_free_list)
  plot_cc(d_centers, cc_work_list, cc_free_list)


if (__name__ == "__main__"):
  run(input_mtz="28molrepEdited_5_refine_001.mtz")

# millerArrays RfactorByResolutionBin

- Read in a phenix.refine mtz file. It plots the work and free R-factors by resolution bin.
- The $ below marks a site for editing.
- This script reads in a phenix.refine mtz file.
- It plots the R-factor by resolution bin.
- The plots are made with matplotlib using miller arrays.
- It also plots the correlation coefficients.
- The plots were made with matplotlib.

This script was adapted from an example script in iotbx:  

Source:  https://github.com/cctbx/cctbx_project/blob/master/iotbx/examples/recalculate_phenix_refine_r_factors.py


```python
# get_ipython().run_line_magic("matplotlib", "inline")

from __future__ import absolute_import, division, print_function
from iotbx.reflection_file_utils import get_r_free_flags_scores
from iotbx.file_reader import any_file
import matplotlib
import matplotlib.pyplot as plt



def compute_r_factors(fobs, fmodel, flags):
  fmodel, fobs = fmodel.common_sets(other=fobs)
  fmodel, flags = fmodel.common_sets(other=flags)
  fc_work = fmodel.select(~(flags.data()))
  fo_work = fobs.select(~(flags.data()))
  fc_test = fmodel.select(flags.data())
  fo_test = fobs.select(flags.data())
  r_work = fo_work.r1_factor(fc_work)
  r_free = fo_test.r1_factor(fc_test)
  
  print("r_work = %.4f" % r_work)
  print("r_free = %.4f" % r_free)
  print("")

  binner = flags.setup_binner(n_bins=20)
  d_star_power = 1.618034
  centers = binner.bin_centers(d_star_power)
  d_centers = list(centers**(-1 / d_star_power))
#   for i in d_centers:
#     print(i)
    
  fo_work.use_binning_of(flags)
  fc_work.use_binner_of(fo_work)
  fo_test.use_binning_of(fo_work)
  fc_test.use_binning_of(fo_work)

  r_work_list = []
  r_free_list = []
  cc_work_list = []
  cc_free_list = []
  for i_bin in fo_work.binner().range_all():
    sel_work = fo_work.binner().selection(i_bin)
    sel_test = fo_test.binner().selection(i_bin)
    fo_work_bin = fo_work.select(sel_work)
    fc_work_bin = fc_work.select(sel_work)
    fo_test_bin = fo_test.select(sel_test)
    fc_test_bin = fc_test.select(sel_test)
    if fc_test_bin.size() == 0 : continue
        
    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,
      assume_index_matching=True)
    r_work_list.append(r_work_bin)
    
    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,
      assume_index_matching=True)
    r_free_list.append(r_free_bin)
    
    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()
    cc_work_list.append(cc_work_bin)
    
    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()
    cc_free_list.append(cc_free_bin)
    
    legend = flags.binner().bin_legend(i_bin, show_counts=False)
    print("%s  %8d %8d  %.4f %.4f  %.3f %.3f" % (legend, fo_work_bin.size(),
      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))
    
  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list


def plot_r_factors(d_centers, r_work_list, r_free_list):
  plt.scatter(d_centers, r_work_list, label=r"$\mathit{R_{work}}$")
  plt.scatter(d_centers, r_free_list, label=r"$\mathit{R_{free}}$")
  plt.xlabel(r"Resolution ($\mathrm{\AA}$)")
  plt.ylabel(r"R-factor (%)")
  plt.legend(loc="upper right")
  plt.savefig("Rs.pdf")
  plt.close()


def plot_cc(d_centers, cc_work_list, cc_free_list):
  plt.scatter(d_centers, cc_work_list, label=r"$\mathit{CC_{work}}$")
  plt.scatter(d_centers, cc_free_list, label=r"$\mathit{CC_{free}}$")
  plt.xlabel(r"Resolution ($\mathrm{\AA}$)")
  plt.ylabel(r"Correlation Coefficeint Fo vs Fc (%)")
  plt.legend(loc="lower right")
  plt.savefig("CCs.pdf")


def run(input_mtz):
  mtz_in = any_file(input_mtz)
  ma = mtz_in.file_server.miller_arrays
  flags = fmodel = fobs = None
  # select the output arrays from phenix.refine.  This could easily be modified
  # to handle MTZ files from other programs.
  for array in ma :
    labels = array.info().label_string()
    if labels.startswith("R-free-flags"):
      flags = array
    elif labels.startswith("F-model"):
      fmodel = abs(array)
    elif labels.startswith("F-obs-filtered"):
      fobs = array
  if (None in [flags, fobs, fmodel]):
    raise RuntimeError("Not a valid phenix.refine output file")
  scores = get_r_free_flags_scores([flags], None)
  test_flag_value = scores.test_flag_values[0]
  flags = flags.customized_copy(data=flags.data()==test_flag_value)

  (d_centers, 
   r_work_list, 
   r_free_list, 
   cc_work_list, 
   cc_free_list) = compute_r_factors(fobs, fmodel, flags)
  plot_r_factors(d_centers, r_work_list, r_free_list)
  plot_cc(d_centers, cc_work_list, cc_free_list)


if (__name__ == "__main__"):
  run(input_mtz="${1:28molrepEdited_5_refine_001}.mtz")
```

In [ ]:
# get_ipython().run_line_magic("matplotlib", "inline")

from __future__ import absolute_import, division, print_function
from iotbx.reflection_file_utils import get_r_free_flags_scores
from iotbx.file_reader import any_file
import matplotlib
import matplotlib.pyplot as plt



def compute_r_factors(fobs, fmodel, flags):
  fmodel, fobs = fmodel.common_sets(other=fobs)
  fmodel, flags = fmodel.common_sets(other=flags)
  fc_work = fmodel.select(~(flags.data()))
  fo_work = fobs.select(~(flags.data()))
  fc_test = fmodel.select(flags.data())
  fo_test = fobs.select(flags.data())
  r_work = fo_work.r1_factor(fc_work)
  r_free = fo_test.r1_factor(fc_test)
  
  print("r_work = %.4f" % r_work)
  print("r_free = %.4f" % r_free)
  print("")

  binner = flags.setup_binner(n_bins=20)
  d_star_power = 1.618034
  centers = binner.bin_centers(d_star_power)
  d_centers = list(centers**(-1 / d_star_power))
#   for i in d_centers:
#     print(i)
    
  fo_work.use_binning_of(flags)
  fc_work.use_binner_of(fo_work)
  fo_test.use_binning_of(fo_work)
  fc_test.use_binning_of(fo_work)

  r_work_list = []
  r_free_list = []
  cc_work_list = []
  cc_free_list = []
  for i_bin in fo_work.binner().range_all():
    sel_work = fo_work.binner().selection(i_bin)
    sel_test = fo_test.binner().selection(i_bin)
    fo_work_bin = fo_work.select(sel_work)
    fc_work_bin = fc_work.select(sel_work)
    fo_test_bin = fo_test.select(sel_test)
    fc_test_bin = fc_test.select(sel_test)
    if fc_test_bin.size() == 0 : continue
        
    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,
      assume_index_matching=True)
    r_work_list.append(r_work_bin)
    
    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,
      assume_index_matching=True)
    r_free_list.append(r_free_bin)
    
    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()
    cc_work_list.append(cc_work_bin)
    
    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()
    cc_free_list.append(cc_free_bin)
    
    legend = flags.binner().bin_legend(i_bin, show_counts=False)
    print("%s  %8d %8d  %.4f %.4f  %.3f %.3f" % (legend, fo_work_bin.size(),
      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))
    
  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list


def plot_r_factors(d_centers, r_work_list, r_free_list):
  plt.scatter(d_centers, r_work_list, label=r"$\mathit{R_{work$")
  plt.scatter(d_centers, r_free_list, label=r"$\mathit{R_{free$")
  plt.xlabel(r"Resolution ($\mathrm{\AA$)")
  plt.ylabel(r"R-factor (%)")
  plt.legend(loc="upper right")
  plt.savefig("Rs.pdf")
  plt.close()


def plot_cc(d_centers, cc_work_list, cc_free_list):
  plt.scatter(d_centers, cc_work_list, label=r"$\mathit{CC_{work$")
  plt.scatter(d_centers, cc_free_list, label=r"$\mathit{CC_{free$")
  plt.xlabel(r"Resolution ($\mathrm{\AA$)")
  plt.ylabel(r"Correlation Coefficeint Fo vs Fc (%)")
  plt.legend(loc="lower right")
  plt.savefig("CCs.pdf")


def run(input_mtz):
  mtz_in = any_file(input_mtz)
  ma = mtz_in.file_server.miller_arrays
  flags = fmodel = fobs = None
  # select the output arrays from phenix.refine.  This could easily be modified
  # to handle MTZ files from other programs.
  for array in ma :
    labels = array.info().label_string()
    if labels.startswith("R-free-flags"):
      flags = array
    elif labels.startswith("F-model"):
      fmodel = abs(array)
    elif labels.startswith("F-obs-filtered"):
      fobs = array
  if (None in [flags, fobs, fmodel]):
    raise RuntimeError("Not a valid phenix.refine output file")
  scores = get_r_free_flags_scores([flags], None)
  test_flag_value = scores.test_flag_values[0]
  flags = flags.customized_copy(data=flags.data()==test_flag_value)

  (d_centers, 
   r_work_list, 
   r_free_list, 
   cc_work_list, 
   cc_free_list) = compute_r_factors(fobs, fmodel, flags)
  plot_r_factors(d_centers, r_work_list, r_free_list)
  plot_cc(d_centers, cc_work_list, cc_free_list)


if (__name__ == "__main__"):
  run(input_mtz="28molrepEdited_5_refine_001.mtz")

# millerArrays dstarsVsMeasurability

- Generate the list of dstars and measurability as lists for plotting by matplotlib.
- The $ below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("${1:3hz7}.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)

Iobs = miller_arrays[1]
# Set up the bins
n_bins = 50
binner = Iobs.setup_binner(n_bins=n_bins)
# binner.show_summary()
used = list(binner.range_used())
selections = [binner.selection(i) for i in used]

# make d_centers for the x-axis
d_star_power = 1.618034
centers = binner.bin_centers(d_star_power)
d_centers = list(centers**(-1 / d_star_power))

# make list of the measurabilities by resolution bin
meas = [Iobs.select(sel).measurability() for sel in selections]

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["figure.dpi"] = 600

fig, ax = plt.subplots(figsize=[3.25, 2.])
ax.scatter(d_centers,lnmeans,c="k",alpha=0.3,s=5.5)

ax.set_xlim(8, 1.5) # decreasing time
ax.set_xlabel(r"$d^*$ in $\AA$",fontsize=12)
ax.set_ylabel("ln(I)",fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(False)
plt.savefig("${1:3hz7}measureability.pdf",bbox_inches="tight")
plt.show()
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("3hz7.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)

Iobs = miller_arrays[1]
# Set up the bins
n_bins = 50
binner = Iobs.setup_binner(n_bins=n_bins)
# binner.show_summary()
used = list(binner.range_used())
selections = [binner.selection(i) for i in used]

# make d_centers for the x-axis
d_star_power = 1.618034
centers = binner.bin_centers(d_star_power)
d_centers = list(centers**(-1 / d_star_power))

# make list of the measurabilities by resolution bin
meas = [Iobs.select(sel).measurability() for sel in selections]

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["figure.dpi"] = 600

fig, ax = plt.subplots(figsize=[3.25, 2.])
ax.scatter(d_centers,lnmeans,c="k",alpha=0.3,s=5.5)

ax.set_xlim(8, 1.5) # decreasing time
ax.set_xlabel(r"$d^*$ in $\AA$",fontsize=12)
ax.set_ylabel("ln(I)",fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(False)
plt.savefig("3hz7measureability.pdf",bbox_inches="tight")
plt.show()

# millerArrays dstarsLogMeans

- Generate the list of dstars and logMeans as lists for plotting by matplotlib.
- The $ below marks a site for editing.

```python
used = list(binner.range_used())
selections = [binner.selection(i) for i in used]

# make means of the intensities by bin
means = [Iobs.select(sel).mean() for sel in selections]
from math import log
lnmeans = [log(y) for y in means]

# meansBR = [Iobs.bijvoet_ratios().select(sel).mean() for sel in selections]

# make d_centers
d_star_power = 1.618034
centers = binner.bin_centers(d_star_power)
d_centers = list(centers**(-1 / d_star_power))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["figure.dpi"] = 600

fig, ax = plt.subplots(figsize=[3.25, 2.])
ax.scatter(d_centers,lnmeans,c="k",alpha=0.3,s=5.5)

ax.set_xlim(${1:8, 1.5}) # decreasing time
ax.set_xlabel(r"$d^*$ in $\AA$",fontsize=12)
ax.set_ylabel("ln(I)",fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(False)
plt.savefig("${2:3hz7}iobsvsdstar.pdf",bbox_inches="tight")
plt.show()

```

# eigenValues 

- The commands to find the eigenvalues and eigenvectors on a tensor. 
- The code is from a post to cctbxbb on 10 December 2020 by Richard Gildea in a reply to Robert Oeffner about code in cctbx for finding eigenvalues and eigenvectors. Robert was requesting the analog in cctbx to scipy.linalg.eig.
- The $ below marks a site for editing.

```python
from scitbx.array_family import flex;
from scitbx.linalg import eigensystem;
m = flex.double((${1:-2, -4, 2, -2, 1, 2, 4, 2, 5}));
m.reshape(flex.grid(3,3));
es = eigensystem.real_symmetric(m);
list(es.values());
list(es.vectors());
```

In [ ]:
from scitbx.array_family import flex;
from scitbx.linalg import eigensystem;
m = flex.double((-2, -4, 2, -2, 1, 2, 4, 2, 5));
m.reshape(flex.grid(3,3));
es = eigensystem.real_symmetric(m);
list(es.values());
list(es.vectors());

# iotbx  

- Read mtz file into a mtz object and print summary.
- The $ below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="/Users/blaine/${1:3nd4}.mtz")
mtz_obj.show_summary()
```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="/Users/blaine/3nd4.mtz")
mtz_obj.show_summary()

# install CCTBX in PyMOL

- Install cctbx inside PyMOL. This protocol worked on a Mac OS.
- The $ below marks a site for editing.

```python
# At the PyMOL prompt in the PyMOL GUI paste the following:
conda install conda-forge::cctbx-base conda-forge::jupyter 
conda install conda-forge::jupyterlab=2.2.0
conda install conda-forge::cctbx-base
# In a terminal
/Applications/PyMOL.app/Contents/bin/jupyter serverextension enable --py jupyterlab --user
/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol --user
/Applications/PyMOL.app/Contents/bin/pip install jupyterlab-snippets-multimenus
/Applications/PyMOL.app/Contents/bin/jupyter lab build
/Applications/PyMOL.app/Contents/bin/jupyter lab clean
/Applications/PyMOL.app/Contents/bin/jupyter --path # select the top option under Data for storing the libraries
cd ~.local/share/jupyter # change as per output from prior line
mkdir multimenus_snippets
cd multimenus_snippets
git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx
git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+
git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol
git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+
/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol37 --user
```

In [ ]:
# At the PyMOL prompt in the PyMOL GUI paste the following:
conda install conda-forge::cctbx-base conda-forge::jupyter 
conda install conda-forge::jupyterlab=2.2.0
conda install conda-forge::cctbx-base
# In a terminal
/Applications/PyMOL.app/Contents/bin/jupyter serverextension enable --py jupyterlab --user
/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol --user
/Applications/PyMOL.app/Contents/bin/pip install jupyterlab-snippets-multimenus
/Applications/PyMOL.app/Contents/bin/jupyter lab build
/Applications/PyMOL.app/Contents/bin/jupyter lab clean
/Applications/PyMOL.app/Contents/bin/jupyter --path # select the top option under Data for storing the libraries
cd ~.local/share/jupyter # change as per output from prior line
mkdir multimenus_snippets
cd multimenus_snippets
git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx
git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+
git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol
git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+
/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol37 --user

# mmtbx makeMaps

- Read in mtz and pdb file and write map coefficients to a separate mtz file.
- The $ below marks a site for editing.

```python
from mmtbx.maps.utils import create_map_from_pdb_and_mtz
'''The phenix.maps commandline tool is the recommended approach.'''
id="${1:3nd4}"
create_map_from_pdb_and_mtz(
          pdb_file="%s.pdb" % id,
          mtz_file="%s.mtz" % id,
          output_file="%s_maps.mtz" % id,
          fill=False,
          out=None,
          llg_map=False,
          remove_unknown_scatterering_type=True,
          assume_pdb_data=False)
```

In [ ]:
from mmtbx.maps.utils import create_map_from_pdb_and_mtz
'''The phenix.maps commandline tool is the recommended approach.'''
id="3nd4"
create_map_from_pdb_and_mtz(
          pdb_file="%s.pdb" % id,
          mtz_file="%s.mtz" % id,
          output_file="%s_maps.mtz" % id,
          fill=False,
          out=None,
          llg_map=False,
          remove_unknown_scatterering_type=True,
          assume_pdb_data=False)